In [ ]:
!{'python3 -m pip install dataflows==0.0.58'}
!{'python3 -m pip install -e git+https://github.com/OriHoch/dataflows-xlsx.git@master#egg=dataflows-xlsx'}

In [3]:
import getpass
CKAN_API_KEY = getpass.getpass("CKAN_API_KEY")

CKAN_API_KEY ····································


In [4]:
CKAN_URL = 'https://odata.org.il'

In [12]:
import requests
from dataflows import Flow, printer, checkpoint

def get_datasets():
    for dataset_id in requests.get(CKAN_URL+'/api/3/action/package_list', headers={'Authorization': CKAN_API_KEY}).json()['result']:
        yield {'id': dataset_id}

Flow(
    get_datasets(),
    checkpoint('all_dataset_ids'),
    printer(tablefmt='html', num_rows=1)
).process()

using checkpoint data from .checkpoints/all_dataset_ids


#,id (string)
1,002b3d2e-179f-4aaa-bd3d-8b2437128e35
2,01-01-2017-01-08-2018
...,
2202,zor-c


(<datapackage.package.Package at 0x7f585f00ec50>, {})

In [22]:
def get_dataset_details():
    datasets = Flow(checkpoint('all_dataset_ids')).results()[0][0]
    for i, dataset in enumerate(datasets):
        dataset = requests.get(CKAN_URL+'/api/3/action/package_show?id='+dataset['id'], headers={'Authorization': CKAN_API_KEY}).json()['result']
        dataset = {
            k: dataset[k] for k in dataset if k in [
                'private', 'num_tags', 'id', 'state', 'type', 'num_resources', 
                'tags', 'groups', 'license_id', 'organization', 'name', 'isopen', 'notes', 'title'
            ]
        }
        dataset['tags'] = [tag['display_name'] for tag in dataset['tags']]
        dataset['groups'] = [group['display_name'] for group in dataset['groups']]
        dataset['organization'] = dataset['organization']['title']
        yield dataset
        if i % 50 == 0:
            print('{} / {}'.format(i, len(datasets)))

Flow(
    get_dataset_details(),
    checkpoint('all_dataset_details'),
    printer(tablefmt='html', num_rows=1)
).process()

saving checkpoint to: .checkpoints/all_dataset_details
using checkpoint data from .checkpoints/all_dataset_ids
0 / 2202
50 / 2202


100 / 2202
150 / 2202
200 / 2202
250 / 2202
300 / 2202
350 / 2202
400 / 2202
450 / 2202
500 / 2202
550 / 2202
600 / 2202
650 / 2202
700 / 2202
750 / 2202
800 / 2202
850 / 2202
900 / 2202
950 / 2202
1000 / 2202
1050 / 2202
1100 / 2202
1150 / 2202
1200 / 2202
1250 / 2202
1300 / 2202
1350 / 2202
1400 / 2202
1450 / 2202
1500 / 2202
1550 / 2202
1600 / 2202
1650 / 2202
1700 / 2202
1750 / 2202
1800 / 2202
1850 / 2202
1900 / 2202
1950 / 2202
2000 / 2202
2050 / 2202
2100 / 2202
2150 / 2202
2200 / 2202


#,private (integer),num_tags (integer),id (string),state (string),type (string),num_resources (integer),tags (array),groups (array),license_id (string),organization (string),name (string),isopen (integer),notes (string),title (string)
1,False,11,19c72fd7-0d3e-4130-8fbc-9d71aa5b0dc9,active,dataset,3,"['2015', '2016', '2017', '2018', 'מצלמות', 'מצלמות אבטחה', 'מרחב ציבורי', 'נהלים', 'עיריית פתח תקווה ...",['עיריית פתח תקוה'],cc-by,עמותת הצלחה,002b3d2e-179f-4aaa-bd3d-8b2437128e35,True,"מענה לבקשת מידע אודות מצלמות במרחב הציבורי, בכלל זה התבקשו מיקומי המצלמות, הנחיות ונהלים המצויות בעי ...",מצלמות המרחב הציבורי- עיריית פתח תקווה
2,False,6,4ad7b057-6dd5-4c1a-aabc-6059b39ef349,active,dataset,12,"['2017', '2018', 'facebook data', 'היחידה הממשלתית לחופש המידע', 'לשנים 2017-2018', 'נתוני פייסבוק']",['משרד המשפטים'],cc-by,משרד המשפטים,01-01-2017-01-08-2018,True,"נתוני הפעילות בפייסבוק של היחידה הממשלתית לחופש המידע, לשנים 2017-2018",נתוני פייסבוק facebook data היחידה הממשלתית לחופש המידע
...,,,,,,,,,,,,,,
2202,False,5,8790a2cf-d669-4ddb-a96d-aa0802e472e4,active,dataset,24,"['דב צור', 'יומן', 'עיריית ראשון לציון', 'ראשון לציון', 'רשות מקומית']",['עיריית ראשון לציון'],cc-by,עמותת הצלחה,zor-c,True,"יומן ראש עיריית ראשון לציון, דב צור, לשנים 2013-2014",יומן ראש עיריית ראשון לציון


checkpoint saved: all_dataset_details


(<datapackage.package.Package at 0x7f585eacd320>, {})

In [2]:
from dataflows import Flow, checkpoint, update_resource
from dataflows_xlsx import dump_to_path

Flow(
    checkpoint('all_dataset_details'),
    update_resource('res_1', name='all_datasets', path='all_datasets.xlsx'),
    dump_to_path('data/all_dataset_details', format='xlsx')
).process()

using checkpoint data from .checkpoints/all_dataset_details


(<datapackage.package.Package at 0x7fdcd2e00ba8>,
 {'count_of_rows': 2202,
  'bytes': 2034,
  'hash': 'a2b9ddaa5a2a25b071b1f7332b87c136',
  'dataset_name': None})